In [1]:
from flask import Flask, render_template, request

In [2]:
import numpy as np
import os

In [3]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

In [6]:
filepath =r"C:\Users\sudhir\Desktop\crop diesese1\crop_model.h5"
model = load_model(filepath)
print(model)
print("Model Loaded Successfully")

Model Loaded Successfully


In [8]:
def pred_corn_dieas(corn_plant):
    test_image = load_img(corn_plant, target_size = (256, 256)) # load image 
    print("@@ Got Image for prediction")
    test_image = img_to_array(test_image)/255 # convert image to np array and normalize
    test_image = np.expand_dims(test_image, axis = 0) # change dimention 3D to 4D
    result = model.predict(test_image) # predict diseased palnt or not
    print('@@ Raw result = ', result)
    pred = np.argmax(result, axis=1)
    print(pred)
    if pred==0:
        return "Corn - Blight Diesease", 'Corn-Blight.html'
       
    elif pred==1:
        return "Corn - Common Rust Disease", 'Corn-Common_Rust.html'
    elif pred==2:
        return "Tomato - Gray Leaf Spot Disease", 'Corn-Gray_Leaf_Spot.html'  

    elif pred==3:
        return "Corn - Healthy and Fresh", 'Corn-Healthy.html'
        

In [9]:
# Create flask instance
app = Flask(__name__)

In [10]:
# render index.html page
@app.route("/", methods=['GET', 'POST'])
def home():
        return render_template('index.html')

In [11]:
# get input image from client then predict class and render respective .html page for solution
@app.route("/predict", methods = ['GET','POST'])
def predict():
    if request.method == 'POST':
        file = request.files['image'] # fet input
        filename = file.filename        
        print("@@ Input posted = ", filename)
        
        file_path = os.path.join(r"C:\Users\sudhir\Desktop\crop diesese1\static\upload", 
                                 filename)
        file.save(file_path)

        print("@@ Predicting class......")
        pred, output_page =pred_corn_dieas(corn_plant=file_path)
              
        return render_template(output_page, pred_output = pred, user_image = file_path)


In [ ]:
# For local system & cloud
if __name__ == "__main__":
    app.run(threaded=False,port=8080) 